SML Assignment-2
Sachin Sharma
2021559

Q4
LDA + KNN

In [13]:
from sklearn.datasets import load_iris 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

In [14]:
n_comps = 1
k_neighbours = 5

In [15]:
class LDA:
    def __init__(self, n_comps, X, y) -> None:
        self.n_comps = n_comps
        self.X = X
        self.y = y
        self.n_features = X.shape[1]
        self.classes = np.unique(y)
        self.n_classes = len(self.classes)
        self.W = self.fit()
        print(self.W.shape)


    def get_class_means(self):
        class_vectors = []
        means = []

        for c in self.classes:
            class_vectors.append(self.X[self.y == c])
            means.append(np.mean(class_vectors[-1], axis=0))

        # class_vectors = np.array(class_vectors)
        means = np.array(means)
        return class_vectors, means


    def S_w(self):
        Sw_matrix = np.zeros((self.n_features, self.n_features))
        class_vectors, means = self.get_class_means()

        for i in range(self.n_classes):
            class_i = np.array(class_vectors[i])
            diff = class_i - means[i]
            Sw_matrix += np.dot(diff.T, diff)

        return Sw_matrix
    

    def S_b(self):
        global_mean = np.mean(self.X, axis = 0)
        class_vectors, means = self.get_class_means()
        Sb_matrix = np.zeros((self.n_features, self.n_features))

        for i in range(self.n_classes):
            diff = means[i] - global_mean
            diff = diff[np.newaxis, :]
            n = class_vectors[i].shape[0]
            Sb_matrix += np.dot(diff.T, diff) * n

        return Sb_matrix
    

    def fit(self):
        Sw_matrix_inv = np.linalg.inv(self.S_w())
        Sb_matrix = self.S_b()
        A = np.dot(Sw_matrix_inv, Sb_matrix)

        eigen_values, eigen_vectors = np.linalg.eig(A)
        pairs = [(np.abs(eigen_values[i]), eigen_vectors.T[i]) for i in range(len(eigen_values))]
        pairs = sorted(pairs, key=lambda x: x[0], reverse=True)

        W = np.vstack([pairs[i][1] for i in range(self.n_comps)]).T
        return W


    def transform(self, X):
        X_proj = np.dot(X, self.W)
        return X_proj

In [16]:
data = load_iris()
X, y = data.data, data.target

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [18]:
lda = LDA(n_comps, X_train, y_train)
X_train_lda = lda.transform(X_train)
X_test_lda = lda.transform(X_test)

(4, 1)


In [19]:
knn = KNeighborsClassifier(n_neighbors = k_neighbours)
knn.fit(X_train, y_train)

knn_lda = KNeighborsClassifier(n_neighbors = k_neighbours)
knn_lda.fit(X_train_lda, y_train)

KNeighborsClassifier()

In [20]:
y_train_pred = knn.predict(X_train)
y_test_pred = knn.predict(X_test)

y_train_pred_lda = knn_lda.predict(X_train_lda)
y_test_pred_lda = knn_lda.predict(X_test_lda)

In [21]:
accuracy_train = accuracy_score(y_train, y_train_pred)
print(f'Training Accuracy without LDA: {accuracy_train}')

Training Accuracy without LDA: 0.95


In [22]:
accuracy_train_lda = accuracy_score(y_train, y_train_pred_lda)
print(f'Accuracy with LDA: {accuracy_train_lda}')

Accuracy with LDA: 0.9833333333333333


In [23]:
accuracy_test = accuracy_score(y_test, y_test_pred)
print(f'Accuracy without LDA: {accuracy_test}')

Accuracy without LDA: 0.9666666666666667


In [24]:
accuracy_test_lda = accuracy_score(y_test, y_test_pred_lda)
print(f'Accuracy with LDA: {accuracy_test_lda}')

Accuracy with LDA: 1.0
